In [1]:
print('Hello World')

Hello World


In [9]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import TimeoutException
import re
import time

In [10]:
# 크롬옵션
chrome_options = Options()
# 꺼짐 방지
chrome_options.add_argument('--disable-gpu') 
# 창 크기에 따라 XPATH가 달라져 크기 고정
chrome_options.add_argument("--start-maximized")
# 알림표시 제거
chrome_options.add_argument("--disable-notifications")
# 옵션 적용
driver = webdriver.Chrome(options=chrome_options)

wan_ins_url = '''https://insight.wanted.co.kr/'''
driver.get(wan_ins_url)

In [ ]:
# 기업 검색창 클릭
driver.find_element(By.XPATH, '//*[@id="__next"]/div[1]/nav/div/div[2]/a/div[2]').click()
# 기업 입력
driver.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div/div/form/div/div/input').send_keys(f'{기업명}')
time.sleep(0.5)
# 엔터
driver.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div/div/form/div/div/input').send_keys(Keys.ENTER)

employee = driver.find_element(By.XPATH, '//*[@id="summary"]/div[2]/div[1]/div[2]/div[2]/div[1]/span').text
sales_money = driver.find_element(By.XPATH, '//*[@id="summary"]/div[2]/div[1]/div[3]/div[2]/div[1]/span').text
sector = driver.find_element(By.XPATH, '//*[@id="summary"]/div[2]/div[2]/dl[1]/dd').text
corp_year = driver.find_element(By.XPATH, '//*[@id="summary"]/div[2]/div[2]/dl[2]/dd').text
corp_url = driver.find_element(By.XPATH, '//*[@id="summary"]/div[2]/div[2]/dl[8]/dd').text

In [3]:
wan_res = pd.read_csv('./.gitignore/wanted_data.csv',index_col=0)
wan_res.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1962 entries, 0 to 1961
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   기업명            1962 non-null   object
 1   주소             1962 non-null   object
 2   직무             1962 non-null   object
 3   자격요건           1962 non-null   object
 4   직급             1962 non-null   object
 5   이용하는기술스택/우대사항  1962 non-null   object
 6   해당 페이지 URL     1962 non-null   object
dtypes: object(7)
memory usage: 122.6+ KB


In [30]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
import re
import time

wan_corp_name = wan_res.기업명.apply(lambda x: re.sub(r'[(a-zA-Z)]', '', x))

last_les = pd.DataFrame(
    columns = ['기업명', '사원수', '설립연도', '매출액', '기업 홈페이지 URL ']
    )

# 크롬옵션
chrome_options = Options()
# 꺼짐 방지
chrome_options.add_argument('--disable-gpu') 
# 창 크기에 따라 XPATH가 달라져 크기 고정
chrome_options.add_argument("--start-maximized")
# 알림표시 제거
chrome_options.add_argument("--disable-notifications")
# 옵션 적용
driver = webdriver.Chrome(options=chrome_options)
wan_ins_url = '''https://insight.wanted.co.kr/'''
driver.get(wan_ins_url)





for corp_name in wan_corp_name:

    # 기업 검색창 클릭
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (
                By.XPATH,
                '//*[@id="__next"]/div[1]/nav/div/div[2]/a/div[2]'
                )
                )
                ).click()

    # 기업 입력
    driver.find_element(
        By.XPATH, 
        '//*[@id="__next"]/div[2]/div/div/form/div/div/input'
        ).send_keys(f'{corp_name}')
    time.sleep(1.2)
    
    try:        
    # 엔터
        driver.find_element(
             By.XPATH,
             '//*[@id="__next"]/div[2]/div/div/form/div/div/input'
             ).send_keys(Keys.ENTER)
   
    except driver.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div/div/form/div[2]/div').is_displayed():
        driver.find_element(
            By.XPATH,
            '//*[@id="__next"]/div[2]/div/div/form/div/div/input'
            ).clear()
        continue

    employee = None
    sales_money = None
    sector = None
    corp_year = None
    corp_url = '-'
    
    try:
        # Crawling-data
        employee = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located(
                    (
                        By.XPATH, 
                        '//*[@id="summary"]/div[2]/div[1]/div[2]/div[2]/div[1]/span'
                        )
                        )
                        ).text
        sales_money = re.sub('\s','', driver.find_element(By.XPATH, '//*[@id="summary"]/div[2]/div[2]/dl[4]/dd').text)
        sector = driver.find_element(By.XPATH, '//*[@id="summary"]/div[2]/div[2]/dl[1]/dd').text
        corp_year = driver.find_element(By.XPATH, '//*[@id="summary"]/div[2]/div[2]/dl[2]/dd').text
        corp_url = driver.find_element(By.XPATH, '//*[@id="summary"]/div[2]/div[2]/dl[8]/dd').text
    except NoSuchElementException:
        pass
    except TimeoutException:
        pass
    
    
    # Data concat
    last_temp = pd.DataFrame(
        [corp_name, employee, corp_year, sales_money, corp_url],
        index = ['기업명', '사원수', '설립연도', '매출액', '기업 홈페이지 URL ']
    ).T
    last_les = pd.concat([last_les, last_temp])

    driver.back()
    time.sleep(0.2)


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=122.0.6261.94)
Stacktrace:
	GetHandleVerifier [0x00007FF76997AD22+56930]
	(No symbol) [0x00007FF7698EF622]
	(No symbol) [0x00007FF7697A42E5]
	(No symbol) [0x00007FF769781D4C]
	(No symbol) [0x00007FF7698123F7]
	(No symbol) [0x00007FF769827891]
	(No symbol) [0x00007FF76980BA43]
	(No symbol) [0x00007FF7697DD438]
	(No symbol) [0x00007FF7697DE4D1]
	GetHandleVerifier [0x00007FF769CF6AAD+3709933]
	GetHandleVerifier [0x00007FF769D4FFED+4075821]
	GetHandleVerifier [0x00007FF769D4817F+4043455]
	GetHandleVerifier [0x00007FF769A19756+706710]
	(No symbol) [0x00007FF7698FB8FF]
	(No symbol) [0x00007FF7698F6AE4]
	(No symbol) [0x00007FF7698F6C3C]
	(No symbol) [0x00007FF7698E68F4]
	BaseThreadInitThunk [0x00007FF91493257D+29]
	RtlUserThreadStart [0x00007FF91642AA58+40]


In [17]:
last_les[last_les.기업 홈페이지 URL == '-' , '기업'] = None
last_les

,기업명,사원수,설립연도,매출액,기업 홈페이지 URL
0,젭,None,None,None,None
0,서커스컴퍼니,13명,11년 (2013년 설립),"22억2,380만원",-
0,블로코,26명,9년 (2015년 설립),"22억1,128만원",-


In [13]:
re.sub('\s','', driver.find_element(By.XPATH, '//*[@id="summary"]/div[2]/div[2]/dl[4]/dd').text)

'22억2,380만원'